In [ ]:
from transformers import AutoTokenizer

checkpoint = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
from datasets import load_dataset

data = load_dataset("universal_dependencies", "te_mtg")
data

In [ ]:
def tokenize_and_align_labels(examples):
    #tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    array = []

    for x in (examples["misc"]):
        tokens = []
        for y in x:
            jsonstr = ast.literal_eval(y)
            tokens.append(jsonstr["Translit"])
        array.append(tokens)
    
    tokenized_inputs = tokenizer(array, truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"upos"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_dataset = data.map(tokenize_and_align_labels, batched=True, remove_columns=data["train"].column_names)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForTokenClassification, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForTokenClassification.from_pretrained(checkpoint)

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

label_names = data["train"].features["upos"].feature.names
id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

label_names = data["train"].features["upos"].feature.names
model = AutoModelForTokenClassification.from_pretrained(checkpoint, id2label=id2label, label2id=label2id)

In [ ]:
training_args = TrainingArguments(
    output_dir="./xlm-roberta-base-te-transliterate",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()
trainer.save_model('./xlm-roberta-base-te') #For reuse

In [ ]:
access_token = "XY#################"
from huggingface_hub import login 

login(token=access_token)


In [ ]:
trainer.push_to_hub()